In [1]:
import ee

ee.Initialize()

In [2]:
dataset = ee.ImageCollection('NOAA/GFS0P25').filterDate('2021-01-01', '2021-01-03')

In [3]:
wind = dataset.select('u_component_of_wind_10m_above_ground')

In [4]:
# get the number of year of data
def datetime(image):
    return ee.Feature(None, {'date': image.date().millis()}) # can be based on .format('YYYY-MM-DD HH-MM-SS')

# get the list of years as a python list
dates = wind.map(datetime).distinct('date').aggregate_array('date').getInfo()
dates = [int(i) for i in dates]
dates

[1609459200000,
 1609480800000,
 1609502400000,
 1609524000000,
 1609545600000,
 1609567200000,
 1609588800000,
 1609610400000]

In [5]:
# a function to reduce/aggregate based on time
def reduceByHour(m):
    hh = int(ee.Date(m).format('HH').getInfo())
    dd = int(ee.Date(m).format('DD').getInfo())
    t1 = wind.filter(ee.Filter.calendarRange(dd, dd, 'day_of_month'))
    t2 = t1.filter(ee.Filter.calendarRange(hh, hh, 'hour')).select(0)
    return t2.mean().set('date', m)

In [6]:
# convert python list to a gee list
hours = ee.List(dates)

hours.getInfo()

[1609459200000,
 1609480800000,
 1609502400000,
 1609524000000,
 1609545600000,
 1609567200000,
 1609588800000,
 1609610400000]

In [7]:
hrw = hours.map(reduceByHour)

EEException: ValueNode is empty